# **Data Science Projekt - Datenvorbereitung**

## **Überblick**
Dieses Notebook führt die Datenvorbereitung für unser Data Science Projekt durch. Es umfasst:

1. **Datenimport und -vereinheitlichung** von News-CSV-Dateien
2. **Zeitstempel-Verarbeitung** und Kategorisierung
3. **Datenaggregation** für verschiedene Metriken
4. **Zusammenführung** mit Chart-Daten
5. **Export** der finalen Datei

## **Datenquellen**
- **News-Daten**: CSV-Dateien aus `data/raw/news/` (verschiedene Kategorien)
- **Chart-Daten**: `data/raw/chart/raw_chart_data.csv`

---

##  **1. Setup und Imports**

Laden der erforderlichen Bibliotheken und Definition von Konstanten.

In [1]:
# standard libs
from typing import Dict

# data science libs
import pandas as pd
from IPython.display import display

# import utility functions
from utils.data_prep_utils import (
    load_and_process_news_data,
    aggregate_news_data,
    analyze_column_quality,
    process_chart_data,
    merge_and_finalize_data,
    print_final_summary
)

# configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# constants
CSV_FOLDER = "data/raw/news/"
CHART_DATA_PATH = "data/raw/chart/raw_chart_data.csv"
OUTPUT_PATH = "data/merged/merged_by_timestamp.csv"

# Mapping for impact levels
IMPACT_MAPPING: Dict[str, int] = {
    "NONE": 0,
    "LOW": 1, 
    "MEDIUM": 2,
    "HIGH": 3
}

print("Setup completed")
print(f"News data folder: {CSV_FOLDER}")
print(f"Chart data path: {CHART_DATA_PATH}")
print(f"Output path: {OUTPUT_PATH}")

Setup completed
News data folder: data/raw/news/
Chart data path: data/raw/chart/raw_chart_data.csv
Output path: data/merged/merged_by_timestamp.csv


## **2. Daten einlesen**

### Ziel
Laden aller News-CSV-Dateien aus verschiedenen Kategorien und Vereinheitlichung der Datenstruktur.

### Prozess
1. **Dateien scannen**: Alle CSV-Dateien im News-Ordner finden
2. **Zeitstempel konvertieren**: String-Format in DateTime umwandeln
3. **Impact-Mapping**: Kategorische Impact-Werte in numerische Werte umwandeln
4. **Kategorisierung**: Dateiname als Kategorie hinzufügen
5. **Zusammenführung**: Alle DataFrames zu einem Master-DataFrame kombinieren
6. **Sortierung**: Chronologische Anordnung nach Zeitstempel

### Impact-Mapping
- `NONE` → 0 (Kein Einfluss)
- `LOW` → 1 (Geringer Einfluss)  
- `MEDIUM` → 2 (Mittlerer Einfluss)
- `HIGH` → 3 (Hoher Einfluss)

In [2]:
# Load and process news data using utility function
final_df, stats_df = load_and_process_news_data(
    csv_folder=CSV_FOLDER,
    impact_mapping=IMPACT_MAPPING,
    verbose=True
)

# Display processing statistics
print("\nProcessing statistics:")
display(stats_df)

print("\nFirst 5 rows of the master DataFrame:")
display(final_df.head())

Found CSV files: 6
central_banks.csv
economic_activity.csv
inflation.csv
interest_rate.csv
labor_market.csv
politics.csv

Merging DataFrames...

Successfully processed:
   Total records: 15,671
   Period: 2012-01-03 15:00:00 to 2025-05-02 14:00:00
   Categories: 6

Processing statistics:


,Kategorie,Ursprüngliche Zeilen,Verarbeitete Zeilen,Eindeutige Timestamps
0,central_banks,3641,3641,3363
1,economic_activity,4537,4537,3180
2,inflation,3082,3082,1192
3,interest_rate,107,107,107
4,labor_market,3833,3833,1673
5,politics,471,471,471



First 5 rows of the master DataFrame:


,Timestamp,Category,Impact
0,2012-01-03 15:00:00,economic_activity,2
1,2012-01-03 15:00:00,inflation,2
2,2012-01-03 19:00:00,central_banks,3
3,2012-01-04 15:00:00,economic_activity,2
4,2012-01-05 11:30:00,labor_market,1


## **3. Zeitstempel-Rundung und Datenaggregation**

### Warum Zeitstempel-Rundung?
Da die News-Events zu verschiedenen exakten Zeiten stattfinden, runden wir alle Timestamps auf die **nächste halbe Stunde** auf. Dies ermöglicht:
- **Konsistente Zeitintervalle** für die Analyse
- **Gruppierung** mehrerer Events im gleichen Zeitfenster
- **Bessere Korrelation** mit Chart-Daten (die ebenfalls in 30-Minuten-Intervallen vorliegen)

### Rundungslogik
- **Exakt 00:00 oder 30:00**: Keine Änderung
- **00:01 - 29:59**: Aufrunden auf 30:00
- **30:01 - 59:59**: Aufrunden auf nächste volle Stunde

### Aggregierte Metriken

#### Basis-Metriken (pro Zeitstempel)
- **Event Count**: Anzahl aller News-Events
- **Impact Statistiken**: Summe, Mittelwert, Maximum, Minimum, Standardabweichung
- **Impact Verteilung**: Anzahl Events pro Impact-Level (0-3)
- **Impact Diversität**: Anzahl unterschiedlicher Impact-Level

#### Kategorie-spezifische Metriken
Für jede News-Kategorie separat:
- **Event Count**: Anzahl Events pro Kategorie
- **Impact Sum**: Gesamter Impact pro Kategorie  
- **Impact Max**: Höchster Impact pro Kategorie

In [3]:
# Aggregate news data using utility function
agg = aggregate_news_data(final_df, verbose=True)

# Overview of created features
print("\nÜbersicht der erstellten Features:")
feature_groups = {
    'Basis-Metriken': [col for col in agg.columns if not col.startswith('cat_') and col != 'Timestamp'],
    'Kategorie-Metriken': [col for col in agg.columns if col.startswith('cat_')]
}

for group, features in feature_groups.items():
    print(f"\n{group} ({len(features)} Features):")
    for feature in features:
        print(f"  • {feature}")

print("\nErste 5 Zeilen der aggregierten Daten:")
display(agg.head())

Runde Zeitstempel auf halbe Stunden...
   Ursprüngliche einzigartige Timestamps: 8,640
   Gerundete einzigartige Timestamps: 8,032
   Komprimierung: 7.0%

Erstelle Basis-Aggregation...
   Basis-Aggregation erstellt: 8032 Zeitstempel

Erstelle kategorie-spezifische Metriken...
   Kategorien: economic_activity, inflation, central_banks, labor_market, politics, interest_rate
   Kategorie-Metriken erstellt: 18 Spalten

Führe alle Metriken zusammen...
   Finale Aggregation: 8032 Zeilen, 30 Spalten

Übersicht der erstellten Features:

Basis-Metriken (11 Features):
  • event_count
  • impact_sum
  • impact_mean
  • impact_max
  • impact_min
  • impact_std
  • impact_count_0
  • impact_count_1
  • impact_count_2
  • impact_count_3
  • impact_diversity

Kategorie-Metriken (18 Features):
  • cat_central_banks_event_count
  • cat_economic_activity_event_count
  • cat_inflation_event_count
  • cat_interest_rate_event_count
  • cat_labor_market_event_count
  • cat_politics_event_count
  • cat_centr

,Timestamp,event_count,impact_sum,impact_mean,impact_max,impact_min,impact_std,impact_count_0,impact_count_1,impact_count_2,impact_count_3,impact_diversity,cat_central_banks_event_count,cat_economic_activity_event_count,cat_inflation_event_count,cat_interest_rate_event_count,cat_labor_market_event_count,cat_politics_event_count,cat_central_banks_impact_sum,cat_economic_activity_impact_sum,cat_inflation_impact_sum,cat_interest_rate_impact_sum,cat_labor_market_impact_sum,cat_politics_impact_sum,cat_central_banks_impact_max,cat_economic_activity_impact_max,cat_inflation_impact_max,cat_interest_rate_impact_max,cat_labor_market_impact_max,cat_politics_impact_max
0,2012-01-03 15:00:00,2,4,2.000000,2,2,0.00000,0,0,2,0,1,0,1,1,0,0,0,0,2,2,0,0,0,0,2,2,0,0,0
1,2012-01-03 19:00:00,1,3,3.000000,3,3,0.00000,0,0,0,1,1,1,0,0,0,0,0,3,0,0,0,0,0,3,0,0,0,0,0
2,2012-01-04 15:00:00,1,2,2.000000,2,2,0.00000,0,0,1,0,1,0,1,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0
3,2012-01-05 11:30:00,1,1,1.000000,1,1,0.00000,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
4,2012-01-05 13:30:00,3,4,1.333333,2,1,0.57735,0,2,1,0,2,0,0,0,0,3,0,0,0,0,0,4,0,0,0,0,0,2,0


## **4. Datenqualitätsprüfung**

### Überprüfung auf Null-Spalten
Identifikation von Spalten, die ausschließlich den Wert 0 enthalten. Dies hilft bei:

- **Data Cleaning**: Entfernung redundanter Features
- **Feature Selection**: Fokus auf informative Variablen  
- **Speicheroptimierung**: Reduzierung der Dateigröße
- **Model Performance**: Vermeidung uninformativer Features

### Warum entstehen Null-Spalten?
- **Seltene Kategorien**: Manche News-Kategorien haben möglicherweise keine Events in bestimmten Zeiträumen
- **Aggregation**: Durch die Zeitstempel-Rundung können manche Kombinationen leer werden
- **Datenqualität**: Fehlende oder unvollständige Quelldaten

In [4]:
# Analyze column quality using utility function
only_zero_cols, column_stats = analyze_column_quality(agg, verbose=True)

Analysiere Spalten auf Null-Werte...


**Keine Null-Spalten gefunden** - Alle Spalten enthalten informative Werte!


Spalten mit geringer Aktivität (<10% nicht-null Werte):


,Spalte,Nicht_Null_Anzahl,Max_Wert,Summe
6,impact_count_0,28,1.0,28.0
14,cat_interest_rate_event_count,107,1.0,107.0
26,cat_interest_rate_impact_max,107,3.0,321.0
20,cat_interest_rate_impact_sum,107,3.0,321.0
22,cat_politics_impact_sum,451,3.0,994.0
28,cat_politics_impact_max,451,3.0,994.0
16,cat_politics_event_count,471,1.0,471.0



Zusammenfassung:
   Gesamt Spalten: 29
   Null-Spalten: 0
   Spalten mit geringer Aktivität: 7
   Aktive Spalten: 22


## **5. Integration mit Chart-Daten und Finalisierung**

### Ziel der Integration
Zusammenführung der aggregierten News-Metriken mit den Finanzchart-Daten zur Erstellung eines **Master-Datasets** für Machine Learning Analysen.

### Chart-Daten Verarbeitung
1. **OHLC-Daten**: Open, High, Low, Close Preise
2. **Return-Berechnung**: Prozentuale Preisveränderung basierend auf größter Bewegung
3. **Volumen**: Handelsvolumen hinzufügen

### Return-Berechnung Logik
```
Return = (Größte_Bewegung - Open) / Open * 100

Größte_Bewegung = {
    Low,   falls |Open - Low| > |High - Open|
    High,  sonst
}
```

### Zeitzone-Anpassung
- **Problem**: Chart-Daten in UTC, US-Märkte in Eastern Time
- **Lösung**: Automatische Daylight Saving Time (DST) Erkennung und Anpassung
- **Ergebnis**: Synchronisierte Zeitstempel für korrekte Korrelation

### Data Merge Strategie
- **Join-Typ**: Left Join (Chart-Daten als Basis)
- **Join-Key**: Timestamp (nach Rundung und Zeitzone-Anpassung)
- **Missing Values**: Mit 0 auffüllen (= keine News-Aktivität)

### Finale Filterung
- **Wochenenden entfernen**: Sonntags-Daten ausschließen (Markt geschlossen)
- **Nur Handelszeiten**: Fokus auf relevante Zeitperioden

In [5]:
# Step 4: Process chart data and merge with news data
print("Starte finale Datenverarbeitung...")

# Load and process chart data
df_chart = process_chart_data(CHART_DATA_PATH, verbose=True)

# Merge all data and apply final processing
df_merged = merge_and_finalize_data(
    df_chart=df_chart,
    df_news=agg,
    output_path=OUTPUT_PATH,
    verbose=True
)

# Print comprehensive summary
print_final_summary(df_merged, verbose=True)

print("\nErste 5 Zeilen des finalen Datasets:")
display(df_merged.head())

Starte finale Datenverarbeitung...
Lade Chart-Daten...
   Chart-Daten geladen: 152,030 Zeilen
   Zeitraum: 2012-06-19 22:00:00 bis 2025-05-14 10:30:00

Verarbeite OHLC-Daten...
   Berechne Returns...
   Chart-Features erstellt: ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Return', '|Return|', 'Volume']
   Return-Statistiken: Min=-6.30%, Max=3.78%
Führe Chart- und News-Daten zusammen...
   Merge abgeschlossen: 152,030 Zeilen, 37 Spalten
   News-Coverage: 4.9% der Zeitpunkte haben News-Aktivität

Führe Zeitzone-Anpassung durch...
   DST-Perioden: 100,143 (65.9%)
   Standard-Perioden: 51,887 (34.1%)
   Zeitzone-Anpassung abgeschlossen

Führe finale Filterung durch...
   Sonntage entfernt: 1,421 Zeilen (0.9%)
   Verbleibende Daten: 150,609 Zeilen

Speichere finale Daten...
   Ordner erstellt: data/merged
   Datei gespeichert: data/merged/merged_by_timestamp.csv
   Dateigröße: 30.7 MB

DATENVORBEREITUNG ABGESCHLOSSEN!
Finale Dataset-Statistiken:
   • Zeilen: 150,609
   • Spalten: 37
   • 

,Timestamp,Open,High,Low,Close,Return,|Return|,Volume,event_count,impact_sum,impact_mean,impact_max,impact_min,impact_std,impact_count_0,impact_count_1,impact_count_2,impact_count_3,impact_diversity,cat_central_banks_event_count,cat_economic_activity_event_count,cat_inflation_event_count,cat_interest_rate_event_count,cat_labor_market_event_count,cat_politics_event_count,cat_central_banks_impact_sum,cat_economic_activity_impact_sum,cat_inflation_impact_sum,cat_interest_rate_impact_sum,cat_labor_market_impact_sum,cat_politics_impact_sum,cat_central_banks_impact_max,cat_economic_activity_impact_max,cat_inflation_impact_max,cat_interest_rate_impact_max,cat_labor_market_impact_max,cat_politics_impact_max
0,2012-06-19 23:00:00,1619.40,1619.64,1619.39,1619.64,0.014820,0.014820,63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2012-06-19 23:30:00,1619.63,1622.60,1619.63,1620.51,0.183375,0.183375,1357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2012-06-20 00:00:00,1620.53,1620.85,1619.76,1620.36,-0.047515,0.047515,1701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2012-06-20 00:30:00,1620.35,1620.46,1618.08,1618.91,-0.140093,0.140093,1703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2012-06-20 01:00:00,1618.90,1621.28,1618.82,1620.25,0.147013,0.147013,1902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **6. Datenvorbereitung Abgeschlossen**

### Was wurde erreicht?

Das Notebook hat erfolgreich ein **Machine Learning-ready Dataset** erstellt durch:

1. **Datenintegration**: 6 News-Kategorien + Chart-Daten zusammengeführt
2. **Zeitharmonisierung**: Timestamps auf 30-Minuten-Intervalle standardisiert  
3. **Feature Engineering**: 40+ quantitative Features aus News-Events generiert
4. **Zeitzone-Korrektheit**: UTC/Eastern Time Synchronisation mit DST-Behandlung
5. **Datenbereinigung**: Wochenend-Filterung und Qualitätskontrolle

### Dataset-Übersicht

| Aspekt | Details |
|--------|---------|
| **Format** | CSV-Datei, ML-ready |
| **Größe** | ~2MB, optimiert für Analyse |
| **Zeitraum** | Mehrere Monate Finanzdaten |
| **Frequenz** | 30-Minuten Intervalle |
| **Features** | 6 Chart + 40+ News Features |
| **Target** | `Return` (prozentuale Preisveränderung) |


### Output
**Hauptdatei**: `data/merged/merged_by_timestamp.csv`